https://towardsdatascience.com/hog-histogram-of-oriented-gradients-67ecd887675f

https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_hog.html

https://www.thepythoncode.com/article/hog-feature-extraction-in-python

In [ ]:
pip install scikit-image

In [ ]:
import matplotlib.pyplot as plt
from skimage import io
from skimage import color
from skimage.transform import resize
import math
from skimage.feature import hog
import numpy as np
import pandas as pd
from PIL import Image
from skimage import data, exposure
import pathlib

In [ ]:
df = pd.read_csv('data/train_complete.csv')

In [ ]:
#image = io.imread("data/train_images/000418bfc.jpg")
image = io.imread("data/train_images/0030401a5.jpg")


In [ ]:
image.shape

In [ ]:
plt.imshow(image)
plt.xlim(0,1600)
plt.ylim(0,256)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

The hog() function takes 6 parameters as input:

image: The target image you want to apply HOG feature extraction.

orientations: Number of bins in the histogram we want to create, the original research paper used 9 bins so we will pass 9 as orientations.

pixels_per_cell: Determines the size of the cell, as we mentioned earlier, it is 8x8.

cells_per_block: Number of cells per block, will be 2x2 as mentioned previously. ==> used for normalisation (16*16)

visualize: A boolean whether to return the image of the HOG, we set it to True so we can show the image.

multichannel: We set it to True to tell the function that the last dimension is considered as a color channel, instead of spatial.

In [ ]:
#without resized picture

fd, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True, channel_axis=-1)

fig, (ax1, ax2) = plt.subplots(2,1, figsize=(32, 16), sharex=True, sharey=True)

ax1.axis('off')
ax1.imshow(image, cmap=plt.cm.gray)
ax1.set_title('Input image')

# Rescale histogram for better display
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

ax2.axis('off')
ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
ax2.set_title('Histogram of Oriented Gradients')
plt.show()

In [ ]:
resized_img = resize(image, (64,128))

In [ ]:
#with resized picture

fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True, channel_axis=-1)

fig, (ax1, ax2) = plt.subplots(2,1, figsize=(32, 16), sharex=True, sharey=True)

ax1.axis('off')
ax1.imshow(resized_img, cmap=plt.cm.gray)
ax1.set_title('Input image')

# Rescale histogram for better display
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

ax2.axis('off')
ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
ax2.set_title('Histogram of Oriented Gradients')
plt.show()

In [ ]:
# fd is Feature Matrix
fd.shape

In [ ]:
hog_image

https://programmerwiki.com/article/9173629512/

SCHRITT 1: VERARBEITEN SIE DIE DATEN VOR (64X128)

Wir müssen das Bild vorverarbeiten und das Seitenverhältnis auf 1: 2 reduzieren. Die Bildgröße beträgt vorzugsweise 64 x 128. Dies liegt daran, dass wir das Bild in 8 teilen8 und 1616 kleine Blöcke zum Extrahieren von Features. Tatsächlich ist dies genau der Wert, der im Originalpapier verwendet wird.

SCHRITT 2: BERECHNEN SIE DEN GRADIENTEN (X- UND Y-RICHTUNG)

Um nun den Gradienten in x-Richtung zu bestimmen, müssen wir den linken Wert vom rechten Pixelwert subtrahieren. Um den Gradienten in y-Richtung zu berechnen, subtrahieren wir den Pixelwert unten vom Pixelwert des ausgewählten Pixels.
Dieser Prozess liefert zwei neue Matrizen, von denen eine den Gradienten in x-Richtung und die andere den Gradienten in y-Richtung speichert.Wir haben die Gradienten in x- und y-Richtung getrennt berechnet. Wiederholen Sie den gleichen Vorgang für alle Pixel im Bild.

SCHRITT 3: BERECHNEN SIE DIE AMPLITUDE UND RICHTUNG

Anhand des Gradienten, den wir im letzten Schritt berechnet haben, bestimmen wir nun die Größe und Richtung jedes Pixelwerts. Für diesen Schritt verwenden wir den Satz von Pythagoras um die Gesamtgradientengröße zu berechnen.
Berechnen Sie als Nächstes die Richtung desselben Pixels. Wir wissen, dass wir den Winkel als tan schreiben können.
Nun haben wir für jeden Pixelwert den Gesamtgradienten (Größe) und die Richtung. Wir müssen diese Gradienten und Richtungen verwenden, um ein Histogramm zu erstellen.

SCHRITT 4: BERECHNEN SIE DAS HISTOGRAMM DES GRADIENTEN IN DER 8 × 8-EINHEIT (9 × 1)

x-Achse: Anzahl der gewählten Bins (beziehen sich auf Winkel); Anzahl entspricht Parameter: "Orientations"
y-Achse: Hier wird Größe des Pixelwertes (Amplitude) anteilig auf die jeweils zum Winkel benachbarten bins eingetragen. Bei 9 bins sind Abstände der bins =20. Wenn Winkel bspw. 36 dann werden (40-36)/20= 4/20 * Amplitudenwert auf bin mit Wert beginnend ab 40 eingetragen und (36-20)/20=16/20 * Amplitudenwert auf bin mit Wert ab 20.

SCHRITT 4: BERECHNEN SIE DAS HISTOGRAMM DES GRADIENTEN IN DER 8 × 8-EINHEIT (9 × 1)

Das im HOG-Feature-Deskriptor erstellte Histogramm wird nicht für das gesamte Bild generiert. Das Bild wird in 8 × 8 Zellen unterteilt und das Histogramm des Richtungsgradienten jeder Zelle berechnet.
Auf diese Weise erhalten wir kleine Blockmerkmale (oder Histogramme), die das gesamte Bild darstellen. Wir können diesen Wert natürlich von 8 x 8 auf 16 x 16 oder 32 x 32 ändern.
Wenn wir das Bild in 8 × 8-Zellen teilen und ein Histogramm erstellen, generieren wir die Matrix mit der im vorherigen Abschnitt beschriebenen Methode 4 und erhalten für jede Zelle eine 9 x 1-Matrix.

SCHRITT 5: NORMALISIEREN SIE DEN GRADIENTEN IN DER 16 × 16-EINHEIT (36 × 1).

Obwohl wir HOG-Funktionen für die 8 × 8-Einheiten des Bildes erstellt haben, ist der Gradient des Bildes sehr empfindlich für die Gesamtbeleuchtung. Dies bedeutet, dass für ein bestimmtes Bild einige Teile des Bildes im Vergleich zu anderen Teilen sehr hell sind.
Wir können diese Änderung der Beleuchtung jedoch reduzieren, indem wir 16 × 16 Blöcke verwenden, um den Gradienten zu normalisieren.
Hier kombinieren wir vier 8 × 8-Einheiten, um einen 16 × 16-Block zu erstellen. Und wir wissen bereits, dass jede 8 × 8-Zelle eine 9 × 1-Matrix für das Histogramm hat. Daher haben wir vier 9 × 1-Matrizen oder eine 36 × 1-Matrix. Um die Matrix zu normalisieren, teilen wir jeden dieser Werte durch die Quadratwurzel der Summe der Quadrate der Werte. Mathematisch für einen gegebenen Vektor V.

SCHRITT 6: MERKMALE DES GESAMTEN BILDES

Wir werden 105 (7 × 15) Blöcke von 16 × 16 haben. Jeder dieser 105 Blöcke hat einen 36 × 1-Vektor als Merkmal. Daher betragen die Gesamtmerkmale des Bildes 105 × 36 × 1 = 3780 Merkmale.

https://medium.com/swlh/histogram-of-oriented-gradients-hog-for-multiclass-image-classification-and-image-recommendation-cf0ea2caaae8

https://www.kaggle.com/anirbanmalick/image-classification-using-hog-knn-98-acc

## Create HoG for all images in train_images

In [ ]:
# get current working directory
cwd = pathlib.Path.cwd()
train_data_dir = cwd.joinpath('data', 'train_images')

In [ ]:
train_images = list(train_data_dir.glob('*.jpg'))

In [ ]:
hog_images = []
hog_features = []
Image_Ids = []
for image in train_images:
    Image_Ids.append(image.name)
    image = io.imread("data/train_images/"+image.name)
    resized_img = resize(image, (64,128))
#    blur = cv.GaussianBlur(image,(5,5),0)
    fd,hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8,8),cells_per_block=(2,2),visualize=True,channel_axis=-1)
    hog_images.append(hog_image)
    hog_features.append(fd)
  
hog_features = np.array(hog_features)

hog_features.shape


#,block_norm= 'L2'

In [ ]:
#for img in hog_images[:20]:
 #   plt.imshow(img)
 #   plt.show()

In [ ]:
hog_features= pd.DataFrame(hog_features)
hog_features

In [ ]:
Image_Ids= pd.DataFrame(Image_Ids)
Image_Ids.rename(columns={0: 'ImageId'}, inplace=True)
Image_Ids

In [ ]:
hog_complete = pd.concat([hog_features, Image_Ids], axis=1, ignore_index=False)
hog_complete

In [ ]:
hog_complete.to_csv('data/train_HOG.csv', sep=',',index=False)

## Create HoG for all images in train_single_defects_augmented

In [ ]:
# get current working directory
cwd = pathlib.Path.cwd()
train_data_dir2 = cwd.joinpath('data', 'augmentations')

In [ ]:
train_images2 = list(train_data_dir2.glob('*.jpg'))

In [ ]:
hog_images_augmented = []
hog_features_augmented = []
Image_Ids2 = []
for image in train_images2:
    Image_Ids2.append(image.name)
    image = io.imread("data/augmentations/"+image.name)
    resized_img = resize(image, (64,128))
#    blur = cv.GaussianBlur(image,(5,5),0)
    fd2,hog_image_augmented = hog(resized_img, orientations=9, pixels_per_cell=(8,8),cells_per_block=(2,2),visualize=True,channel_axis=-1)
    hog_images_augmented.append(hog_image_augmented)
    hog_features_augmented.append(fd2)
  
hog_features_augmented = np.array(hog_features_augmented)

hog_features_augmented.shape


#,block_norm= 'L2'

In [ ]:
hog_features_augmented= pd.DataFrame(hog_features_augmented)
hog_features_augmented

In [ ]:
Image_Ids2= pd.DataFrame(Image_Ids2)
Image_Ids2.rename(columns={0: 'ImageId'}, inplace=True)
Image_Ids2

In [ ]:
hog_complete_augmented = pd.concat([hog_features_augmented, Image_Ids2], axis=1, ignore_index=False)
hog_complete_augmented

In [ ]:
hog_complete_augmented.to_csv('data/train_HOG_augmented.csv', sep=',',index=False)